In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: canaanbreiss1 (lm-informants). Use `wandb login --relogin` to force relogin


True

In [2]:
import pandas as pd 
import wandb
import numpy as np
from tqdm import tqdm


api = wandb.Api(timeout=60)
entity, project = "lm-informants", "0717_english_sweep"  # set to your entity and project 
runs = api.runs(entity + "/" + project) 

In [3]:
df = pd.DataFrame()

In [4]:

summary_list, config_list, name_list = [], [], []
mega_df = pd.DataFrame()

all_data = []

for run in tqdm(runs): 
    
    # add all the keys that are logged that you want to download
    keys = [
        #"strategy_used_is_train",
        "step",
        "auc",
        ]
                
    df = pd.DataFrame()
    
    history = run.scan_history()
    
    history_df = pd.DataFrame(history)
                
    # filter ones that were killed
    if run.state != "finished":
        print("filtering run: ", run.path)
        continue
        
    # get the experiment config
    config = {k: v for k,v in run.config.items()
         if not k.startswith('_')}
    
    exp = {}
    
    # if key doesn't exist, set to nan (though this shouldn't happen after filtering empty runs)
    exp.update({f"{col}": history_df[~history_df[col].isnull()][col].values if col in history_df.columns else np.nan for col in keys})

    num_steps = len(exp[keys[0]])
    for k in keys:
        assert len(exp[k]) == num_steps
        
    # convert dict of lists to list of dicts
    results = [dict(zip(exp,t)) for t in zip(*exp.values())]
    
    for r in results:
        r.update({f"config/{key}": val for key, val in config.items()})
        r.update({f"strategy": run.name})
        r.update({f"wandb_id": run.path})
        
    all_data.extend(results)
    #print("extended!")
    
    
# get status
    
mega_df = pd.DataFrame(all_data)


  2%|█▋                                                                                 | 1/48 [00:00<00:29,  1.62it/s]

filtering run:  ['lm-informants', '0717_english_sweep', '7opcoaqo']


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:57<00:00,  1.20s/it]


In [5]:
display(mega_df)

,step,auc,config/run,config/n_init,config/strategy,config/tolerance,config/prior_prob,config/warm_start,config/feature_type,config/lexicon_file,config/converge_type,config/num_candidates,config/log_log_alpha_ratio,config/max_updates_observe,config/max_updates_propose,strategy,wandb_id
0,0.0,0.507315,0,0,train,0.000002,0.001280,False,english,None,symmetric,50,3.638863,NaN,NaN,train,"[lm-informants, 0717_english_sweep, f2kl5xkh]"
1,1.0,0.535915,0,0,train,0.000002,0.001280,False,english,None,symmetric,50,3.638863,NaN,NaN,train,"[lm-informants, 0717_english_sweep, f2kl5xkh]"
2,2.0,0.562551,0,0,train,0.000002,0.001280,False,english,None,symmetric,50,3.638863,NaN,NaN,train,"[lm-informants, 0717_english_sweep, f2kl5xkh]"
3,3.0,0.574500,0,0,train,0.000002,0.001280,False,english,None,symmetric,50,3.638863,NaN,NaN,train,"[lm-informants, 0717_english_sweep, f2kl5xkh]"
4,4.0,0.610624,0,0,train,0.000002,0.001280,False,english,None,symmetric,50,3.638863,NaN,NaN,train,"[lm-informants, 0717_english_sweep, f2kl5xkh]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9395,195.0,0.618299,0,0,unif,0.000002,0.001385,False,english,None,symmetric,50,-0.582270,1.0,1.0,unif,"[lm-informants, 0717_english_sweep, 8snsz3gp]"
9396,196.0,0.618943,0,0,unif,0.000002,0.001385,False,english,None,symmetric,50,-0.582270,1.0,1.0,unif,"[lm-informants, 0717_english_sweep, 8snsz3gp]"
9397,197.0,0.619597,0,0,unif,0.000002,0.001385,False,english,None,symmetric,50,-0.582270,1.0,1.0,unif,"[lm-informants, 0717_english_sweep, 8snsz3gp]"
9398,198.0,0.620227,0,0,unif,0.000002,0.001385,False,english,None,symmetric,50,-0.582270,1.0,1.0,unif,"[lm-informants, 0717_english_sweep, 8snsz3gp]"


In [6]:
out_path = './runs_english_sweep.csv'
mega_df.to_csv(out_path)